## Aufgabe 1

### 8-Queens Problem

Das Spielfeld wird in einem Vektor mit 8 Dimensionen dargestellt, wobei gilt:

#### Kodierung

$$
Z \in \mathbb{N} := \{x \in \mathbb{N} \mid 0 <= x <= 7\}
$$

$$
V = (Z_1, Z_2, Z_3, Z_4, Z_5, Z_6, Z_7, Z_8)
$$

Der Index des Vektors ist die X-Koordinate und der Wert ist die Y-Koordinate des Spielfelds. Dadurch lässt sich das Spielfeld abbilden.

#### Operatoren

Crossover: Es werden zwei Individuen ausgewählt, basierend auf ihrer Fitnessfunktion, und an einer zufälligen Stelle gespalten und zusammengeführt. Für die Auswahl der Individuen wird die Glücksrad methode verwendet. Bessere individuen werden häufiger in dem Rad vertreten durch diese funktion $f(x)=(8-x)^2$ wobei x der fittniswert des individums ist.

Mutation: Nach dem Crossover hat jedes neue Individuum die Chance, einen zufälligen Wert durch einen anderen zufälligen Wert zu ersetzen.

#### Fitness-Funktion

Bei der Fitness-Funktion prüft jede Dame, ob sie geschlagen werden kann. Die Anzahl der Damen, die geschlagen werden können, ist der Wert der Funktion.

Das bedeutet: Je geringer der Wert, desto besser das Individuum.

#### Simulated Annealing

???

### Karte färben

#### Kodierung

Für die Länder die aneinander gränzen wird eine statisches dictionary benutzt, das als key den index des landes hat und als value ein Array mit den index der Länder die angränzen.

die Individuen werden in einem Array mit der länder der länder gespeichert.Jedem Land wird eine index im Array zugewiesen.
In dem Array wird die Farbe gespeichert mit der das Land eingefärbt ist.

die Farben werden in einem Seperatem Array gespeichert.
#### Operatoren
Crossover: Es werden zwei Individuen ausgewählt, basierend auf ihrer Fitnessfunktion, und an einer zufälligen Stelle gespalten und zusammengeführt. 

Mutation: Nach dem Crossover hat jedes neue Individuum die Chance, die Farbe eines zufälligen landes,  durch eine andere zufälligen Farbe zu ersetzen.

#### Fitniss-Funktion
Für die Fitniss werden die länder gezählt die ein nachbarland mit der selben Farbe haben.

## Aufgabe 2
### 8-Queens Problem

In [8]:
import random
import pandas as pd

class Board:
    def __init__(self, positions):
        self.positions = positions

    def fitniss(self):

        marked = [[False]*8 for _ in range(8)]
        for i in range(8):
            x = i
            y = self.positions[i]
            for j in range(8):
                if (j != y):
                    marked[x][j] = True
                if (j != x):
                    marked[j][y] = True
            
            temp_x = x
            temp_y = y
            for _ in range(8):
                temp_x = temp_x - 1
                temp_y = temp_y - 1
                if (temp_x < 0 or temp_y < 0):
                    break
                marked[temp_x][temp_y] = True

            temp_x = x
            temp_y = y
            for _ in range(8):
                temp_x = temp_x + 1
                temp_y = temp_y - 1
                if (temp_x > 7 or temp_y < 0):
                    break
                marked[temp_x][temp_y] = True

            temp_x = x
            temp_y = y
            for _ in range(8):
                temp_x = temp_x - 1
                temp_y = temp_y + 1
                if (temp_x < 0 or temp_y > 7):
                    break
                marked[temp_x][temp_y] = True
            
            temp_x = x
            temp_y = y
            for _ in range(8):
                temp_x = temp_x + 1
                temp_y = temp_y + 1
                if (temp_x > 7 or temp_y > 7):
                    break
                marked[temp_x][temp_y] = True
                
        counter = 0
        for i in range(8):
            x = i
            y = self.positions[i]
            if (marked[x][y]):
                counter = counter + 1
        return counter

    def mutate(self,mutate_chance):
        mutate_pool = random.randint(1,1000)

        if (mutate_pool <= mutate_chance):
            mutate_index = random.randint(0,7)
            self.positions[mutate_index] = random.randint(0,7)

    def printBoard(self):
        pos = [[False]*8 for _ in range(8)]
        for i in range(8):
            x = i
            y = self.positions[i]
            pos[x][y] = True

        for row in pos:
            print(" ".join('x' if cell else '_' for cell in row))
        print()

        


def crossover(board1, board2):
    point = random.randint(1,6)
    child1 = board1.positions[:point] + board2.positions[point:]
    child2 = board2.positions[:point] + board1.positions[point:]
    return Board(child1), Board(child2)

def select(boards, numBoardsTotal,mutate_chance):
    new_boards: Board = []
    selection_pool = []
    for i in range(len(boards)-1):
        selection_value = 8-boards[i].fitniss()
        for _ in range(selection_value ** 2):
            selection_pool.append(i)

    for _ in range(numBoardsTotal // 2):
        index1 = random.randint(0,len(selection_pool)-1)
        index2 = random.randint(0,len(selection_pool)-1)

        while(selection_pool[index1] == selection_pool[index2]):
            index2 = random.randint(0,len(selection_pool)-1)

        boardindex1 = selection_pool[index1]
        boardindex2 = selection_pool[index2]

        board1, board2 = crossover(boards[boardindex1], boards[boardindex2])

        board1.mutate(mutate_chance)
        board2.mutate(mutate_chance)
        new_boards.append(board1)
        new_boards.append(board2)

    return new_boards

def Evo(boards, generations, numBoardsTotal,mutate_chance):
    for g in range(generations):
        boards = sorted(boards, key= lambda x: x.fitniss())

        new_boards: Board = select(boards, numBoardsTotal,mutate_chance)


        boards = new_boards

        for b in boards:
            if b.fitniss() == 0:
                return b, g
    
    boards = sorted(boards, key= lambda x: x.fitniss())
    return boards[1], -1

def RunSimulation(numBoardsTotal, generations, mutate_chance):
    boards: Board = []
    for _ in range(numBoardsTotal):
        boards.append(Board([random.randint(0,7) for _ in range(8)]))

    board, generation = Evo(boards, generations, numBoardsTotal,mutate_chance)
    return pd.Series([board.fitniss(),board.positions, generation], index=["fitniss","positionen","generation"])



numBoardsTotal = 10
generations = 10
mutate_chance = 20 # 10 = 1%

results = []
for i in range(100):

    results.append(RunSimulation(numBoardsTotal,generations,mutate_chance))
    print(i)

df = pd.DataFrame(results)

df = df.sort_values("generation",ascending=False)

df.head(50)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


,fitniss,positionen,generation
0,5,"[4, 7, 4, 5, 2, 0, 3, 6]",-1
1,4,"[4, 6, 0, 5, 7, 5, 3, 2]",-1
2,5,"[7, 7, 0, 2, 4, 1, 3, 0]",-1
3,5,"[1, 3, 1, 6, 2, 1, 0, 7]",-1
4,6,"[2, 2, 4, 0, 3, 0, 0, 7]",-1
5,4,"[3, 0, 2, 6, 1, 7, 4, 1]",-1
6,5,"[5, 7, 2, 1, 7, 7, 0, 3]",-1
7,4,"[5, 3, 0, 0, 0, 4, 7, 1]",-1
8,4,"[4, 4, 0, 5, 1, 4, 7, 3]",-1
9,5,"[7, 5, 0, 5, 1, 3, 0, 6]",-1


#### Mit:
numBoardsTotal = 10
generations = 50
mutate_chance = 50

Wurde bei nur einem durchgang in der 32 generation ein ergebniss gefunden.

#### Mit:

numBoardsTotal = 100
generations = 50
mutate_chance = 50

Wurde bei 27 durchläufen ein Ergebniss gefunden. Frühste Generation: 1 Spätiste: 45

#### Mit:
numBoardsTotal = 10
generations = 200
mutate_chance = 50

wurde bei 8 durchläufen ein Ergebniss gefunden. Frühste Generation: 4 Spätiste: 134

#### Mit:
numBoardsTotal = 10
generations = 50
mutate_chance = 200

Wurde bei 13 durchläufen ein Ergebniss gefunden. Frühste Generation: 18 Spätiste: 49

#### Mit:
numBoardsTotal = 10
generations = 50
mutate_chance = 50

Wurde bei 13 durchläufen ein Ergebniss gefunden. Frühste Generation: 18 Spätiste: 49

### Fazit
Den Größten einfluss hat die anzahl der Boards die Paralel in der Simulation aktive sind. Danach hat die Mutations chance den höchsten einfluss. den geringsten einfluss scheint die anzahl der Generationen zu haben.

bei den ersten 3 versuchen war es in der regel so, dass sie sehr für das Ziel gefunden haben oder Garnicht. nur bei der höheren Mutations chance haben auch häufig die späteren generationen zum Erfolg geführt.

## Aufgabe 2
Karte Färben

In [ ]:
import random
import pandas as pd

random.seed()

nachbarn = {
    0:[1,2],
    1:[0,2,3],
    2:[0,1,3],
    3:[1,2,4],
    4:[3],
    5:[],
}

color = ["White", "Black", "Red", "Blue", "Green"]

class Karte:
    def __init__(self, länder):
        self.länder = länder
        pass

    def fitness(self):
        counter = 0
        colors_used = []
        for i in range(len(self.länder)):
            current_col = self.länder[i]
            if current_col not in colors_used:
                colors_used.append(current_col)
            for j in nachbarn[i]:
                if (current_col == self.länder[j]):
                    counter += 1
        counter += len(colors_used)
        
        return counter
    
    def mutate(self, mutate_chance):
        mutate_pool = random.randint(1,1000)

        if (mutate_pool <= mutate_chance):
            mutate_index = random.randint(0,4)
            mutate_index = random.randint(0,4)
            c = color[random.randint(0,4)]
            self.länder[mutate_index] = c

    def printColors(self):
        print(self.länder)
    

def crossover(board1, board2):
    point = random.randint(1,6)
    child1 = board1.länder[:point] + board2.länder[point:]
    child2 = board2.länder[:point] + board1.länder[point:]
    return Karte(child1), Karte(child2)

def select(karten, numBoardsTotal, mutate_chance):
    new_karte: Karte = []
    selection_pool = []
    for i in range(len(karten)-1):
        selection_value = 10-karten[i].fitness()
        for _ in range(selection_value ** 2):
            selection_pool.append(i)

    for _ in range(numBoardsTotal // 2):
        index1 = random.randint(0,len(selection_pool)-1)
        index2 = random.randint(0,len(selection_pool)-1)

        while(selection_pool[index1] == selection_pool[index2]):
            index2 = random.randint(0,len(selection_pool)-1)

        kartenindex1 = selection_pool[index1]
        kartenindex2 = selection_pool[index2]

        karte1, karte2 = crossover(karten[kartenindex1], karten[kartenindex2])

        karte1.mutate(mutate_chance)
        karte2.mutate(mutate_chance)
        new_karte.append(karte1)
        new_karte.append(karte2)

    return new_karte
    
def Evo(karten, generations, numKartenTotal, mutate_chance):
    for g in range(generations):
        karten = sorted(karten, key= lambda x: x.fitness())
        #print(karten[1].fitness())

        new_karten: Board = select(karten, numKartenTotal, mutate_chance)

        karten = new_karten

        for k in karten:
            if k.fitness() <= 3:
                return k,g
    
    karten = sorted(karten, key= lambda x: x.fitness())
    return karten[1],-1

def RunSimulation(numKartenTotal,generations,mutate_chance):
    karten: Karte = []
    for _ in range(numKartenTotal):
        karten.append(Karte([color[random.randint(0,4)] for _ in range(5)]))

    karte, generation = Evo(karten,generations,numKartenTotal, mutate_chance)

    return pd.Series([generation,karte.fitness(),karte.länder], index = ["generation", "fitness", "Länder Farbe"])

numKartenTotal = 4
generations = 4
mutate_chance = 5

results = []
for i in range(100):

    results.append(RunSimulation(numKartenTotal,generations,mutate_chance))
    #print(i)

df = pd.DataFrame(results)

df = df.sort_values("generation",ascending=False)

df.head(50)


,generation,fitness,Länder Farbe
2,2,3,"[White, Blue, Green, White, Blue]"
21,2,3,"[Blue, White, Green, Blue, Green]"
3,1,3,"[Blue, Red, Green, Blue, Green]"
0,1,3,"[Black, Green, Red, Black, Red]"
32,1,3,"[Red, Blue, White, Red, White]"
19,1,3,"[Green, White, Red, Green, White]"
33,1,3,"[Blue, Black, Red, Blue, Black]"
18,1,3,"[White, Blue, Black, White, Black]"
80,1,3,"[White, Black, Blue, White, Blue]"
52,1,3,"[Green, Blue, Red, Green, Blue]"


Bei dem gegebenem problem kann so einfach eine lösung gefunden werden, das bereits bei sehr kleinen werten